# CartPole RL com Q-Learning
**Aluno:** André 
**Data:** 20/05/2025  
**Descrição:** Implementação de Q-Learning para resolver CartPole-v1 sem redes neurais.


## 1. Modelagem como MDP
- **Estados (4):** posição, velocidade do carrinho; ângulo, velocidade angular do pêndulo  
- **Ações (2):** 0 = esquerda, 1 = direita  
- **Recompensa:** +1 a cada passo não terminal  
- **Bellman (Q-Learning):**  
  $$Q(s,a)\leftarrow Q(s,a)+\alpha\bigl[r + \gamma\,\max_{a'}Q(s',a') - Q(s,a)\bigr]$$


In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt


## 2. Discretização de Estados
Usamos 4 dimensões contínuas mapeadas em `n_bins = (6, 12, 6, 12)`.


In [ ]:
# Parâmetros
n_bins = (6, 12, 6, 12)
env = gym.make('CartPole-v1')

# Limites razoáveis para cada dimensão
obs_low  = np.array([-2.4, -3.0, -0.2094, -3.5])
obs_high = np.array([ 2.4,  3.0,  0.2094,  3.5])

# Cria os pontos de corte (bins)
bins = [np.linspace(obs_low[i], obs_high[i], n_bins[i]-1)
        for i in range(len(n_bins))]

def discretize(obs):
    """Retorna tupla de índices discretos para um obs contínuo."""
    state = []
    for i, val in enumerate(obs):
        idx = np.digitize(val, bins[i])
        state.append(idx)
    return tuple(state)


## 3. Inicialização da Q-Table e Hiperparâmetros


In [ ]:
# Q-table com shape = (*n_bins, n_actions)
Q_table = np.zeros(n_bins + (env.action_space.n,))

# Hiperparâmetros
alpha       = 0.1      # learning rate
gamma       = 0.99     # discount factor
epsilon     = 1.0      # exploração inicial
eps_decay   = 0.995
min_epsilon = 0.01
n_episodes  = 10000


## 4. Loop de Treinamento (Q-Learning)


In [ ]:
rewards = []

for ep in range(n_episodes):
    obs = env.reset()
    state = discretize(obs)
    total_reward = 0
    done = False

    while not done:
        # política ε-greedy
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q_table[state])

        obs2, reward, done, _ = env.step(action)
        state2 = discretize(obs2)

        # Q-Learning update
        best_next = np.max(Q_table[state2])
        Q_table[state + (action,)] += alpha * (
            reward + gamma * best_next - Q_table[state + (action,)]
        )

        state = state2
        total_reward += reward

    # decaimento de epsilon
    epsilon = max(min_epsilon, epsilon * eps_decay)
    rewards.append(total_reward)


## 5. Resultados e Gráficos
Média móvel de recompensa (janelas de 100 episódios)


In [ ]:
# Cálculo da média móvel
mov_avg = np.convolve(rewards, np.ones(100)/100, mode='valid')

plt.figure(figsize=(8,4))
plt.plot(mov_avg)
plt.title('Recompensa média (janela=100 eps)')
plt.xlabel('Episódios')
plt.ylabel('Recompensa média')
plt.show()
